In [1]:
%load_ext autoreload
%autoreload 2

import os 
import numpy as n
from matplotlib import pyplot as plt
import napari

In [4]:
# path to wherever the s2p-lbm repository is cloned on your computer
os.chdir(os.path.dirname(os.path.abspath("")))

# set to "false" to run code without messages intended for developers
os.environ["SUITE3D_DEVELOPER"] = "true"

from suite3d.job import Job
from suite3d import ui
from suite3d import io
from suite3d import tiff_utils as tfu

Install gitpython for dev benchmarking to work


In [3]:
## Find all the tiffiles in the tif path
# File I/O is pipelined, so the data doesn't have to be on a fast SSD 
# single HDDs or reasonably fast network drives should work without much difference in speed 
tif_path = r'\\zaru.cortexlab.net\Subjects\ATL020\2023-04-12\701' #C:\Users\andrew\Documents\localData\suite3d\Suite3D Demo Data' # r'/mnt/md0/data/demo'
tifs = io.get_tif_paths(tif_path)

In [4]:
# Set the mandatory parameters
planes = n.arange(5)
params = {
    # volume rate
    'fs': io.get_vol_rate(tifs[0]),
    # planes to analyze. 0 is deepest, 30 is shallowest (corrected for ScanImage channel IDs)
    # you should keep all the planes to do crosstalk estimation! 
    'planes' : planes,
    'crosstalk_n_planes' : len(planes)//2,
    
    # Decay time of the Ca indicator in seconds. 1.3 for GCaMP6s. This example is for GCamP8m
    'tau' : 1.3,
    'lbm' : False, 
    'num_colors' : 2, # if not lbm data, how many color channels were recorded by scanimage
    'functional_color_channel' : 0, # if not lbm data, which color channel is the functional one
    'fuse_strips' : False, # don't do this, it's only needed for LBM data
    'fix_shallow_plane_shift_estimates' : False,
    'subtract_crosstalk' : False, # I think this is unnecessary for non LBM data...
}

### "Job" structure
The unimaginatively named `Job` structure is meant to contain all of the parameters, data, logs, and results for a single recording. It will be created in the root directory provided with the given name. All intermediate and final results will be saved in this directory, so I recommend using a fast SSD for this (and moving results to slow HDD once processing is complete).

All the print statements you see (and more) are also logged in `<job dir>/logfile.txt`. If you want things to look cleaner, reduce the verbosity to 2 (full logs will still be in the logfile).

To load a previously created job (to do more processing or load results), set `create=False`. If `create=True` but there exists another job of the same name in the root directory, it will either overwrite the parameters of the previous job or throw an error (depending on the `overwrite` parameter). Note, overwriting isn't as catastrophic as it sounds since data isn't deleted and remains accessible, but you might lose the saved parameters and some metadata.

In [11]:
# Create the job
job = Job(r'C:\Users\andrew\Documents\localData\suite3d\runs','atl-test', tifs = tifs,
          params=params, create=True, overwrite=True, verbosity = 3)

Job directory C:\Users\andrew\Documents\localData\suite3d\runs\s3d-atl-test already exists
Loading job directory for atl-test in C:\Users\andrew\Documents\localData\suite3d\runs
   Loading dirs 
      Created dir C:\Users\andrew\Documents\localData\suite3d\runs\s3d-atl-test\registered_fused_data
      Found dir summary
      Created dir C:\Users\andrew\Documents\localData\suite3d\runs\s3d-atl-test\iters
   Loading default params
      Updating param fs
      Updating param planes
      Updating param crosstalk_n_planes
      Updating param tau
      Updating param lbm
      Updating param num_colors
      Updating param functional_color_channel
      Updating param fuse_strips
      Updating param fix_shallow_plane_shift_estimates
      Updating param subtract_crosstalk
   Updated main params file


## Initial pass
This pass takes a few files (`n_init_files`, usually ~200-300 frames is enough) and does the following:
- estimates the crosstalk coefficient between the lower set of 15 planes and the higher 15 planes
- computes the shifts between successive planes caused by the xy-shift of the light beads
- estimates the optimal number of pixels that overlap between successive strips, so they can be fused together
- calculates a "reference volume" that will be used later in registration 

In [12]:
# optional parameters for initialization
# load 1 file to initialize
job.params['n_init_files'] = 1
# If set to None, use all of the frames in the loaded init files 
# if your files are really big, set this to <300
job.params['init_n_frames'] = None

# Set to None to auto-compute the crosstalk coefficient
# You can set a float value between 0-1 (usually around 0.1-0.3) to override the calculation
job.params['override_crosstalk'] = None
# number of processors to use
job.params['n_proc_corr'] = 12

In [14]:
%%time
# This step only uses `n_init_files` files, so the  runtime will stay the same even with larger recordings
# soon this will also be gpu-ified to be faster!
job.run_init_pass()

   Saved a copy of params at C:\Users\andrew\Documents\localData\suite3d\runs\s3d-atl-test\summary
   Updated main params file
Launching initial pass
Saving summary to C:\Users\andrew\Documents\localData\suite3d\runs\s3d-atl-test\summary\summary.npy
   Loading init tifs with 30 channels
      Loading \\zaru.cortexlab.net\Subjects\ATL020\2023-04-12\701\2023-04-12_701_ATL020_2P_00001_00009.tif
   Loaded 1 files, total 0.98 GB
   Loaded init tifs
   Loaded movie with 400 frames and shape 5, 512, 512
      Enforcing positivity in mean image
   No crosstalk estimation or subtraction
   Using 3d registration
   Computing plane alignment shifts
   Applying plane alignment shifts


ValueError: cannot assign slice from input of different size

## Registration
First, we do registration over time of the  xy-drift caused by brain movement. This is similar to Suite2P registratrion, it does rigid registration followed by non-rigid registration. This is accelerated on the GPU. Suite2P registration parameters can be changed, see `default_params.py` for a list of all parameters related to registration. After you have registered, you can load the registered fused movie into memory and take a look at the mean image. I suggest cropping the dark edges if you have any as shown in the cells below.

If you run out of gpu memory, try reducing the `gpu_reg_batchsize` parameter. I have a A4500 with 20GB memory which works well with a batchsize of 10.

In [ ]:
# If you have large tiffs, split the large tiffs into files of size 100 after registration
job.params['split_tif_size'] = 100

In [ ]:
%%time
job.register_gpu()

In [ ]:
# if GPU fails, the following *should* work
# note that in the GPU version fusing is incorporated into registration
# job.register()
# job.params['n_skip'] = job.load_summary()['fuse_shift']
# job.fuse_registered_movie()

In [ ]:
mov_full = job.get_registered_movie('registered_fused_data','f')
im_full = mov_full.mean(axis=1).compute()

In [ ]:
# find crop values that minimize dark zones - check planes 0 and 15 in the following cell to 
# make sure you're not cutting out parts of the brain
crop = ((0,18), (100,1100), (50, 900))

In [ ]:
io.show_tif(im_full[0,crop[1][0]:crop[1][1], crop[2][0]:crop[2][1]])
io.show_tif(im_full[len(job.params['planes'])//2,crop[1][0]:crop[1][1], crop[2][0]:crop[2][1]])

## SVD Denoising
We compute an SVD of the volumetric movie, and keep the top N components to denoise. This is done by blocking the volume to make it computationally tractable. The blocks have overlaps, and I find that for noisy movies you will get some grid-like artifacts if your block overlaps aren't set such that each non-edge pixel is included in at least two blocks. So, I usually set the overlaps to be half of the block shape to achieve this. Feel free to try with smaller (or zero) overlaps on your data to see if it works better (overlaps increase the number of blocks that need to be SVD-d, so less overlap = less blocks = faster compute). 

The SVD decomposition is implemented with Dask, which can be blazing fast (compared to other methods) if implemented correctly, but there are a few parameters that can make it really slow if set incorrectly. If the SVD feels slow, try playing with the `svd_pix_chunk` and `svd_time_chunk` parameters. If you really care about speed, probably install the Dask profiler and see if there are any obvious bottlenecks.

Note that the Dask SVD uses an approximate algorithm and its runtime scales **sub-linearly** with movie length! So for a short movie, it might take >10x the movie duration, but for longer movies it should be much less.

In [ ]:
# tag (not path) for the directory where the SVD will be saved
svd_dir_tag = 'svd_cropped'

# Number of components to compute per block 
# (you can change the actual number of components used for computation later, this is the upper limit)
job.params['n_svd_comp'] = 100
# Size of each block in pixels in z,y,x
job.params['svd_block_shape'] = (4,200,200)
# overlap in z,y,x between two neighboring blocks
job.params['svd_block_overlaps'] = (2,100,100)
# crop the movie before computing svd
job.params['svd_crop'] = crop

# Number of pixels in each Dask "chunk" when computing SVD. Unless you have ridiculously 
# large blocks, manually setting the chunksize to the total number of pixels in a block
# seems to be substantially faster than having multiple chunks per block
job.params['svd_pix_chunk'] = n.product(job.params['svd_block_shape'])
# When computing SVD, we can compute multiple blocks (4-8) at the same time, which is sometimes 
# faster since we save on some disk I/O for neighboring blocks (I  think)
# for longer recordings (1000+frames) or if you have issues with RAM, set to 1
job.params['n_svd_blocks_per_batch'] = 8

In [ ]:
%%time
with_svd = False
if with_svd:
    # create the directory where we'll save the SVD, and run the SVD decomposition
    job.make_new_dir(svd_dir_tag)
    svd_info = job.svd_decompose_movie(svd_dir_tag, run_svd=True)

## Calculating the correlation map

The correlation map is the most important part of the cell detection process. It spatially and temporally filters the denoised movie, normalizes it, thresholds it, and accumulates it over time to create a volume where cells should be made more visible and neuropil is removed.

**You should tune some of these parameters for your data**, each described below. To enable easy tuning, there is a **parameter sweep interface** that can try many combinations for a subset of the movie quickly, and visualize the results (you will find this below). 

Correlation map improves the more frames you have!

In [ ]:
# number of SVD components to use when calculating the correlation map
# lower number means more denoising, but if it's too low you will start losing cells!
# This can't be larger than the number of svd components you used in the decomposition above
job.params['n_svd_comp'] = 50

# spatial filter sizes for neuropil subtraction, and cell detection
# npil_filt is a low_pass filter that attempts to remove any features larger than the filter size (neuropil!)
# conv_filt_xy is a high_pass filter that amplifies any features that are smaller than ~2x the filter size (cells!)
# these values worked well for me with ~4um xy pixel spacing and ~15 um z pixel spacing, for detecting mouse somata
# When you change resolution, or if you're trying to detect smaller things, you will need to adjust these values
# because the units here are _pixels_, not microns!
job.params['cell_filt_type'] = 'gaussian'
job.params['cell_filt_xy_um'] = 9.0
job.params['cell_filt_z_um'] = 0.6
job.params['npil_filt_type'] = 'unif'
job.params['npil_filt_xy'] = 25.0
job.params['npil_filt_z']=  2.5

# normalization exponent, should be around 1. 
# If you find blood vessels or the background being too bright in the correlation map, reduce it to ~0.7-0.8! 
job.params['sdnorm_exp']= 0.75

# threshold applied to the normalized, filtered movie before it is accumulated into the correlation map
# if you increase it, the background will become darker (which is good!), however at some point you will
# start excluding dimmer cells (which is bad!)
job.params['intensity_thresh']=0.3

## Compute parameters 
# number of frames to compute at one iteration 
# (any value above ~100-200 shouldn't affect results, 
# decrease if you have RAM issues or if SVD reconstruction gets stuck on "Sending all blocks to dask to compute")
job.params['t_batch_size'] = 300
# number of processors to use when calculating the correlation map
job.params['n_proc_corr'] = 12
# number of frames per smaller batch within the batch, should be ~t_batch_size / n_proc_corr, but above ~5
job.params['mproc_batchsize'] = 5

import multiprocessing
num_cores = multiprocessing.cpu_count()
assert job.params['n_proc_corr'] < num_cores, f"Your computer has {num_cores} but job.params['n_proc_corr'] is set to {job.params['n_proc_corr']}"

In [ ]:
%%time
# uncomment below to load svd_info for and svd you did earlier if you are re-running this notebook
# svd_info = n.load(os.path.join(job.dirs['svd_cropped'], 'svd_info.npy'), allow_pickle=True).item()
if with_svd:
    corrmap = job.calculate_corr_map(mov = svd_info)
else:
    mov_full = job.get_registered_movie('registered_fused_data', 'fused')
    crop = ((0,18), (100,1100), (50, 900))
    mov_crop = mov_full[crop[0][0]:crop[0][1], :, crop[1][0]:crop[1][1], crop[2][0]:crop[2][1]]
    corrmap = job.calculate_corr_map(mov = mov_crop)

### Optional: sweep correlation map parameters

In [ ]:
# Pick the parameters you want to sweep, and enter them in the tuples.
# It will do all combinations of parameters, so if you have many parameters it will be many combinations! 

do_sweep = True
if do_sweep:
    job.params['t_batch_size'] = 300
    params_to_sweep = {
        'intensity_thresh' : (0.3,),
        #'n_svd_comp' : (50,), # if you have multiple values here, make sure you pass mov=svd_info
        'cell_filt_xy_um': (2.0, 8.0, 16.0),
        'npil_filt_xy_um': (5.0, 15.0, 25.0), # (15.0, 20.0, 25.0),
        'cell_filt_z_um' : (0.6,),
        'npil_filt_z_um' : (1.5, ), #(2.0, 3.0),
        'sdnorm_exp' : (0.75,)
    }
    
    sweep_summary = job.sweep_corrmap(params_to_sweep, mov = mov_full, all_combinations=True)

In [ ]:
# use this to access older sweep results
load_old_sweep = False        
if do_sweep and load_old_sweep:
    sweep_summary = n.load(os.path.join(job.dirs['sweep-full'], 'sweep_summary.npy'),allow_pickle=True).item()

        

In [ ]:
# Use this to look at the sweep results
if do_sweep:
    v = job.vis_vmap_sweep(sweep_summary)

In [ ]:
# reminder: you should set all parameters that you swept back to the values you want explicitly before re-computing the correlation map
# because the global values of the parameters are updated during the sweep!

## Detection
Now that we have a correlation map, we can segment the cells. The algorithm is similar to suite2p, it does the following:
1. Choose the peak value on the correlation map, and set this pixel as your _seed cell_
2. Identify the _active frames_ of the candidate cell by taking all frames where the activity of the cell is greater than a threshold (`activity_thresh`) or have activity above a certain percentile (`percentile`) 
3. Look at the activity of all neighboring pixels of the _seed cell_ during the _active frames_ of the seed cell. If a candidate pixel's activity is similar to the activity of the _seed cell_ (similarity above `extend_thresh`), include the pixel in the _seed cell_.
4. Repeat steps 2-3 until you've grown the cell as much as you can
5. Remove the cell from the correlation map
6. Find the next largest peak of the correlation map, take this as your seed cell and repeat steps 2-6
7. Stop when the peak value of the remaining correlation map is below a user-specified threhsold (`peak_thresh`)
Two main improvements over Suite2p: first, this is done in 3D. Second, it is parallelized to be much faster, as it works on patches of the movie separately!

**The most important variable that you *must* set is `peak_thresh`**. To do this, use the cell below to visualize the correlation map (`vmap` for short) using napari. Use the contrast sliders to find a minimum value where all spots above this value look like they might be cells. I find it useful to set the range to be very small, all pixels above the minimum are basically white. You should try to get rid of most of the obvious noise (e.g. artifacts at edges or around blood vessels, specks of single-pixel white spots, stuff outside the brain). It is not critical to exclude everything, you can be generous here and remove ROIs based on other criteria later. However, if you are too generous, you'll end up with too many pixels above the threhsold and your detection will take forever, with a lot of extra junk cells. I recommend starting a little conservative, and then push the limits. 

**Other useful variables**: When you have long enough recordings, 0.2 for `extend_thresh` is OK. However, if you have only a very short recording, or you find many cells that are much larger than they should be (with large, sprinkly footprints that extend way beyond the cell), or you have large cloudy blobs of noise being picked up as cells, increase `extend_thresh`. `activity_thresh` and `percentile` work together, usually it's good enough to just pick one and change it. If you have few frames, or you feel like you have low signal, it's better to set these to be lower, so you include more frames when evaluating a cell. However, if you can afford to, it's good to keep them high (`activity_thresh` around 10, `percentile` around 99.0), because then sparsely-firing cells will be picked up easier. Play around and see!

**To make it faster to try parameters, you can run the detection only on a subset of the patches**. By default the movie is split into ~100 patches (I think), but if you pass `job.patch_and_detect(do_patch_idxs=(10,20,50,80))` then the detection will only run on the specified patches.

**Detection always works better with more frames!** 300 frames is a very small number, so don't expect it to work perfectly on this demo.

In [ ]:
# visualize the correlation map
# here you can identify the best "peak_thresh"
# play with the contrast limits of vmap until only cells are visible
# the lower contrast limit should be used as "peak_thresh"

# as you change the contrast limit for the vmap image, the "viewer status" in the bottom left will print the value for you

results = job.load_corr_map_results()
mean_img = results['mean_img']
vmap = results['vmap']

v = napari.Viewer(title="Identify peak_thresh!")
v_meanimg = v.add_image(mean_img, name='mean image')
v_vmap = v.add_image(vmap, name='vmap')

def print_contrast_limits(event):
    v.status = f"Current peak_thresh: {v_vmap._contrast_limits[0]:.2f}"
    return None
    
_ = v_vmap.events.connect(print_contrast_limits)


In [ ]:
%%time
job.params['detection_timebin'] = 1 
job.params['detection_time_crop'] = (None,None)
job.params['max_pix'] = 250
job.params['peak_thresh'] = 78

job.params['activity_thresh'] = 1.0
job.params['extend_thresh'] = 0.3
job.params['max_iter'] = 10000 # maximum number of ROIs detected in given patch
combined_dir = job.patch_and_detect(combined_name='combined-full') #, do_patch_idxs=(20,30,50,60,80,90))

### Compute neuropil masks, extract activity and deconvolve
For each cell, compute a donut around it excluding all other cells to use it to estimate the local neuropil activity. Then, extract the activity of the cell and the neuropil, subtract 0.7\*neuropil activity from the ROI activity, and deconvolve using Oasis. Make sure you have set the `tau` parameter correctly for the deconvolution.

In [ ]:
job.compute_npil_masks(combined_dir)

In [ ]:
traces = job.extract_and_deconvolve(stats_dir = combined_dir)

## Visualization
Visualize using napari. While the "cells" layer is celected, left click on a cell to see its activity traces. You can right click to manually mark cells as "non-cells", and you can use the sliders to filter cells based on size or the correlation map peak.

In [ ]:
combined_dir = job.dirs['detection-combined-full']

In [ ]:
outputs = ui.load_outputs(combined_dir, load_traces=True)
v,layers = ui.create_ui(outputs)
ui.add_callbacks_to_ui(v,layers,outputs,savedir=combined_dir)

## Load outputs and analyze
This is how you can access the traces for each cell, and the locations for each cell

In [ ]:
outputs = ui.load_outputs(combined_dir, load_traces=True)
print(outputs.keys())

In [ ]:
n_cells, n_t = outputs['F'].shape
frame_times = n.arange(n_t) / outputs['fs']
example_cell = 1200
plt.plot(frame_times, outputs['F'][example_cell], label='ROI Fluorescence')
plt.plot(frame_times, outputs['Fneu'][example_cell], label='Neuropil Fluorescence')
plt.plot(frame_times, outputs['spks'][example_cell], label='Deconvolved activity')
plt.legend()
plt.xlabel("Time (s)")
plt.ylabel("Activity")

In [ ]:
# results of your manual curation will be saved here
# ROIs that are marked as cells will be 1, non-cells 0
iscell = outputs['iscell_curated_slider']

In [ ]:
# suite2p-style list of all cells
cell_stats = outputs['stats']
print(cell_stats[example_cell].keys())

In [ ]:
cell_coords = [stat['coords'] for stat in cell_stats]
cell_centers = n.array([stat['med'] for stat in cell_stats])
vmap_shape = outputs['vmap'].shape

In [ ]:
# make a volume with the footprints of all cells and plot a max-projection of it along z
cell_vol = ui.fill_cells_vol(cell_coords, fill_vals = n.ones(len(cell_coords)), empty=0)
io.show_tif((cell_vol ).max(axis=0))

In [ ]:
hist,bins = n.histogram(cell_centers[:,0],bins = n.arange(vmap_shape[0]))
bins = bins[:-1]

plt.plot(hist, bins)
plt.xlabel("# of neurons at a given depth")
plt.ylabel("Depth from surface (um)")
plt.yticks(n.arange(bins.max(),0,-bins.max()//6), -15*(bins.max()-n.arange(bins.max(),0,-bins.max()//6)));

# from the plot, seems like the shallowest plane has lots of cells, 
# possibly because it's out of the brain and it's mostly noise...
# it might be a good idea to exclude them in the curation 
# probably, when you have a longer recording you won't have this issue as much

### Save a fancy 3D plot
Use UCSF Chimera to open the .mrc file and visualize your cells

In [ ]:
v1, v2 = ui.make_label_vols(outputs['stats'], outputs['vmap'].shape, 
            iscell =  outputs['iscell_curated_slider'], 
                  cmap='Blues', lam_max = 0.3)

In [ ]:
io.save_mrc(combined_dir, 'curated_cells.mrc',v2[:,:,:,3], voxel_size=(4,4,15))